![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/NER_DRUGS_DEVELOPMENT_TRIALS.ipynb)

# **Colab Setup**

In [17]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [18]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

# **Start the Spark session**

In [19]:
import json
import os

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

Spark NLP Version : 3.4.4
Spark NLP_JSL Version : 3.5.2


# **🔎 For about models**

### 📌 **bert_token_classifier_drug_development_trials**

It is a BertForTokenClassification NER model to identify concepts related to drug development including **Trial Groups , End Points , Hazard Ratio and other** entities in free text.

# **🔎Define Spark NLP pipeline**

In [20]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence") 

tokenizer = Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

tokenClassifier = MedicalBertForTokenClassifier.pretrained( "bert_token_classifier_drug_development_trials", "en", 'clinical/models')\
    .setInputCols("sentence","token")\
    .setOutputCol("ner")\
    .setCaseSensitive(True)

ner_converter = NerConverter()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

pipeline =  Pipeline(stages=[
                documentAssembler,
                sentenceDetector, 
                tokenizer, 
                tokenClassifier, 
                ner_converter])

pipelineModel = pipeline.fit(spark.createDataFrame([['']]).toDF("text"))


sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
bert_token_classifier_drug_development_trials download started this may take some time.
[OK!]


# **🔎Sample Text**

In [21]:
sample_text = "In July 2018, final pfs analysis results were reported from 358 patients with relapsed/refractory follicular and marginal zone lymphoma showing that the trial met its primary endpoint, with a significant improvement in pfs compared with rituximab + placebo.   At that time, follow-up was to continue for the mature os results. The median pfs per irc assessment (primary endpoint) with lenalidomide + rituximab and rituximab + placebo was 39.4 and 14.1months, respectively "

data = spark.createDataFrame([[sample_text]]).toDF('text')

In [22]:
result = pipelineModel.transform(data)

# **🔎Run the pipeline**

In [23]:
light_result = LightPipeline(pipelineModel).fullAnnotate(sample_text)


chunks = []
entities = []
sentence= []
begin = []
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence'])
    
    
import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 
                   'sentence_id':sentence, 'entities':entities})

df.head(20)

,chunks,begin,end,sentence_id,entities
0,pfs,20,22,0,End_Point
1,358 patients,60,71,0,Total_Patients
2,pfs,219,221,0,End_Point
3,rituximab + placebo,237,255,0,Trial_Group
4,os,315,316,1,End_Point
5,median,331,336,2,Duration
6,pfs,338,340,2,End_Point
7,lenalidomide + rituximab,385,408,2,Trial_Group
8,rituximab + placebo,414,432,2,Trial_Group
9,39.4,438,441,2,Value


# **🔎Visualize results**

In [24]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()
visualiser.display(result = light_result[0] ,label_col = 'ner_chunk', document_col = 'document')